# Reference bias using expected vs observed # of bases with no coverage

* https://www.illumina.com/documents/products/technotes/technote_coverage_calculation.pdf
* https://en.wikipedia.org/wiki/DNA_sequencing_theory#Early_uses_derived_from_elementary_probability_theory
* http://seqanswers.com/forums/showpost.php?p=161353&postcount=2

The problem can be boiled down to the following question: given sequence coverage $X$, the proportion of sites with no coverage can be modelled as $Poisson(\lambda = 0)$. Under ideal conditions and infinite number of sites, the empirical counts of sites with no coverage should be exactly equal to this value.

What is the proportion of sites with no coverage in our samples? Are there some worrying differences between Neanderthal and Denisovan samples? Keep in mind that both _Denisova 4_ and _Denisova 8_ have almost the same TMRCA, significantly different from all other Neanderthals, who also give the same values between each other.

In [1]:
library(tidyverse)
library(glue)
library(here)
suppressPackageStartupMessages({library(rtracklayer); library(GenomicRanges)})

devtools::load_all(".")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.0
✔ tibble  2.1.1     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘glue’

The following object is masked from ‘package:dplyr’:

    collapse

here() starts at /mnt/expressions/mp/ychr
Warning message:
“no function found corresponding to methods exports from ‘GenomicRanges’ for: ‘concatenateObjects’”Loading ychr


In [2]:
cov_df <- readRDS(here("data/rds/cov_df.rds"))

In [3]:
samples <- unique(cov_df$name)

In [4]:
samples

[1] "a00"               "den4"              "den8"             
 [4] "elsidron1"         "mez2"              "S_BedouinB-1"     
 [7] "S_Burmese-1"       "S_Dai-2"           "S_Dinka-1"        
[10] "S_Finnish-2"       "S_French-1"        "S_Gambian-1"      
[13] "S_Han-2"           "S_Ju_hoan_North-1" "S_Karitiana-1"    
[16] "S_Mandenka-1"      "S_Mbuti-1"         "S_Papuan-2"       
[19] "S_Punjabi-1"       "S_Saami-2"         "S_Sardinian-1"    
[22] "S_Thai-1"          "S_Turkish-1"       "S_Yoruba-2"       
[25] "spy1"              "ustishim"          "shotgun_mez2"     
[28] "shotgun_spy1"      "elsidron2"

In [5]:
avg_cov <- cov_df %>%
    group_by(name, regions) %>%
    summarise(avg_coverage = mean(coverage)) %>%
    spread(regions, avg_coverage) %>%
    rename(avg_cov = full) %>%
    select(-exome, -lippold) %>%
    filter(!str_detect(name, "elsidron"))

In [16]:
avg_cov <- avg_cov %>%
    mutate(observed_zero = round(map_dbl(name, ~ mean(filter(cov_df, name == .x)$coverage == 0)), 5),
           expected_zero = round(dpois(x = 0, lambda = avg_cov), 5))

In [17]:
avg_cov %>%
    mutate(difference = observed_zero - expected_zero) %>% arrange(-difference) %>% print(n=Inf)

# A tibble: 27 x 5
# Groups:   name [27]
   name              avg_cov observed_zero expected_zero difference
   <chr>               <dbl>         <dbl>         <dbl>      <dbl>
 1 spy1                0.910      0.490           0.402    0.0872  
 2 den4                1.66       0.260           0.189    0.0706  
 3 den8                3.60       0.0975          0.0272   0.0702  
 4 mez2               14.6        0.0400          0        0.0400  
 5 shotgun_mez2        0.888      0.434           0.412    0.0222  
 6 shotgun_spy1        0.522      0.611           0.593    0.0172  
 7 S_Mbuti-1          20.6        0.00152         0        0.00152 
 8 S_Yoruba-2         17.2        0.00151         0        0.00151 
 9 S_Mandenka-1       16.5        0.0015          0        0.0015  
10 S_Gambian-1        20.4        0.00149         0        0.00149 
11 ustishim           20.4        0.00068         0        0.00068 
12 a00                21.3        0.00046         0        0.00046 
13 S_Pa